# ETL Pipeline for Data Modeling in Cassandra

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)
    
def query_to_df(session, query):
    return pd.DataFrame(list(session.execute(query)))

## Task 1
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### PK Choice
- sessionId is selected as the partition key because we will have a lot of sessions and will be on the filter item_in_session is the clustering key to maintain the order in the session.
- this garantee we will have one record per item_in_session


In [8]:
query1 = """
SELECT 
    artist_name,
    song_title,
    song_length 
FROM 
    song_by_session 
WHERE 
    session_id = 338 
    and item_in_session = 4;
"""
query1_create_table = """
CREATE TABLE IF NOT EXISTS song_by_session (
    session_id int,
    item_in_session int,
    artist_name text,
    song_title text,
    song_length float,
    PRIMARY KEY ((session_id), item_in_session)
);
"""
try:
    session.execute(query1_create_table)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_session (session_id, item_in_session, artist_name, song_title, song_length) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
query_to_df(session, query1)

,artist_name,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Task 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### PK Choice
- (user_id, session_id) is selected as the partition key because we will have a lot of sessions from lots of users and use both on the filter;
- item_in_session is the clustering key to maintain the order in the session;
- they garantee that we will have one record per item per session and can filter by user too.

In [11]:
query2 = """
SELECT 
    artist_name,
    song_title,
    first_name,
    last_name,
    item_in_session
FROM 
    song_by_user_session
WHERE 
    user_id = 10 
    and session_id = 182;
"""
query2_create_table = """
CREATE TABLE IF NOT EXISTS song_by_user_session (
    user_id int,
    session_id int,
    item_in_session int,
    artist_name text,
    song_title text,
    first_name text,
    last_name text,
    PRIMARY KEY ((user_id, session_id) , item_in_session)
);
"""
try:
    session.execute(query2_create_table)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user_session (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
query_to_df(session, query2)

,artist_name,song_title,first_name,last_name,item_in_session
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
1,Three Drives,Greece 2000,Sylvie,Cruz,1
2,Sebastien Tellier,Kilometer,Sylvie,Cruz,2
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3


## Task 3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### PK Choice
- song_title is selected as the partition key because we will have a good amount of songs to distribute on the partitions and will be use to filter

- user_id is the clustering key to maintain the order of the users 
- both garantee that will we have only one record per user

In [14]:
query3 = """
SELECT 
    first_name,
    last_name
FROM 
    user_by_song 
WHERE 
    song_title = 'All Hands Against His Own'
"""
query3_create_table = """
CREATE TABLE IF NOT EXISTS user_by_song (
    song_title text,
    user_id int,
    session_id int,
    artist_name text,
    item_in_session int,
    first_name text,
    last_name text,
    PRIMARY KEY ((song_title), user_id)
);
"""
try:
    session.execute(query3_create_table)
except Exception as e:
    print(e)
                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song (song_title, user_id, session_id, artist_name, item_in_session, first_name, last_name) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), int(line[8]), line[0], int(line[3]), line[1], line[4]))

In [16]:
query_to_df(session, query3)

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
session.execute("DROP TABLE IF EXISTS song_by_session")
session.execute("DROP TABLE IF EXISTS song_by_user_session")
session.execute("DROP TABLE IF EXISTS user_by_song")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()